# Infer ***golgi*** - part 4️⃣

--------------
## **OBJECTIVE**

### <input type="checkbox"/> Infer sub-cellular component  ***golgi***  
Segment the ***golgi*** from a single channel (membrane marker). This workflow was optimized for images of fluorescently tagged sialyltransferase1 (SiT1), a membrane protein. The Golgi has both the classical cisternae structure as well as small golgi derived vesciles, so we are utilizing two segmentation methods to idenfity the entire Golgi structure.

---------------------
## infer ***golgi complex***
### summary of steps 

➡️ **EXTRACTION**
- **`STEP 1`** - Select a channel for segmentation

    - select single channel containing the golgi marker (channel number = user input)

**PRE-PROCESSING**
- **`STEP 2`** - Rescale and smooth image

  - rescale intensity of composite image (min=0, max=1)
  - median filter (median size = user input)
  - gaussian filter (sigma = user input)

**CORE PROCESSING**
- **`STEP 3`** - Global + local thresholding (AICSSeg – MO)

    - apply MO thresholding method from the Allen Cell [aicssegmentation](https://github.com/AllenCell/aics-segmentation) package (threshold options = user input)

- **`STEP 4`** - Thin segmentation (Topology preserving)

    - apply topology preserved thinning function from the Allen Cell [aicssegmentation](https://github.com/AllenCell/aics-segmentation) package (thinning amount, minimum thickness = user input)

- **`STEP 5`** - ‘Dot’ thresholding method (AICSSeg)

    - apply "dot" thresholding method (for small round objects) from the Allen Cell [aicssegmentation](https://github.com/AllenCell/aics-segmentation) package (size scale and threshold cutoff = user input)

- **`STEP 6`** - Combine Segmentations (logical or)

    - combine the two segmentations with logical *OR*

**POST-PROCESSING**
- **`STEP 7`** - Remove small holes and objects

  - fill holes (hole size = user input)
  - remove small objects (object size = user input)
  - filter method (method = user input)

**POST-POST-PROCESSING**
- **`STEP 8`** - Label objects

  - label unique golgi objects based on connectivity

**EXPORT** ➡️
- save labeled ***golgi*** (golgi, GL) as unsigned integer 16-bit tif files


> Generally following the Allen Cell Segmenter procedure for segmentation of Golgi from the [sialyltransferase 1 (ST6GAL1)](https://www.allencell.org/cell-observations/category/golgi-apparatus) marker. Sourced from: this [script](https://github.com/AllenCell/aics-segmentation/blob/main/aicssegmentation/structure_wrapper/seg_st6gal1.py).

---------------------
## **IMPORTS**

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block loads all of the necessary python packages and functions you will need for this notebook.

In [ ]:
from pathlib import Path
import os

import numpy as np

from aicssegmentation.core.utils import topology_preserving_thinning
import napari
from napari.utils.notebook_display import nbscreenshot

from infer_subc.core.file_io import (read_czi_image,
                                                                    # read_ome_image,
                                                                    export_inferred_organelle,
                                                                    # import_inferred_organelle,
                                                                    list_image_files)

from infer_subc.core.img import *

%load_ext autoreload
%autoreload 2


## **LOAD AND READ IN IMAGE FOR PROCESSING**

#### &#x1F6D1; &#x270D; **User Input Required:**

In [ ]:
# Specify the file type of your raw data that will be analyzed. Ex) ".czi" or ".tiff"
im_type = ".czi"

## Define the path to the directory that contains the input image folder.
data_root_path = Path(os.path.expanduser("~")) / "Documents/Python_Scripts/Infer-subc"

## Specify which subfolder that contains the input data
in_data_path = data_root_path / "raw"

## Specify the output folder
out_data_path = data_root_path / "out"

# Specify which file you'd like to segment from the img_file_list
test_img_n = 5

#### &#x1F3C3; **Run code; no user input required**

In [ ]:
if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

img_file_list = list_image_files(in_data_path,im_type)
# pd.set_option('display.max_colwidth', None)
# pd.DataFrame({"Image Name":img_file_list})

In [ ]:
test_img_name = img_file_list[test_img_n]

img_data,meta_dict = read_czi_image(test_img_name)

channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

# ***EXTRACTION prototype - golgi***

## **`STEP 1` - Select a channel for segmentation**

- select single channel containing the golgi marker (channel number = user input)

In [ ]:
###################
# INPUT
###################
GOLGI_CH = 3
raw_golgi = select_channel_from_raw(img_data, GOLGI_CH)

# ***PRE-PROCESSING prototype - golgi***

## **`STEP 2` - Rescale and smooth image**

- rescale intensity of composite image (min=0, max=1)
- median filter (median size = user input)
- gaussian filter (sigma = user input)

In [ ]:
###################
# PRE_PROCESSING
###################
median_sz = 4
gauss_sig = 1.34

struct_img =  scale_and_smooth(raw_golgi,
                               median_size = median_sz, 
                               gauss_sigma = gauss_sig)

# ***CORE-PROCESSING prototype - golgi***

## **`STEP 3` - Global + local thresholding (AICSSeg – MO)**

- apply MO thresholding method from the Allen Cell [aicssegmentation](https://github.com/AllenCell/aics-segmentation) package (threshold options = user input)

In [ ]:
###################
# CORE_PROCESSING
###################
################################
# segment the majority of the golgi with this global and local thresholding method
thresh_method = 'tri'
cell_wise_min_area = 1200
thresh_adj = 1

bw = masked_object_thresh(struct_img, 
                          global_method=thresh_method, 
                          cutoff_size=cell_wise_min_area, 
                          local_adjust=thresh_adj)

## **`STEP 4` - Thin segmentation (Topology preserving)**

- apply topology preserved thinning function from the Allen Cell [aicssegmentation](https://github.com/AllenCell/aics-segmentation) package (thinning amount, minimum thickness = user input)

In [ ]:
# thin segmentation with maintain a minimal required thickness
thin_dist = 1
thin_dist_preserve = 1.6

bw_thin = topology_preserving_thinning(bw, thin_dist_preserve, thin_dist)

## **`STEP 5` - ‘Dot’ thresholding method (AICSSeg)**

- apply "dot" thresholding method (for small round objects) from the Allen Cell [aicssegmentation](https://github.com/AllenCell/aics-segmentation) package (size scale and threshold cutoff = user input)

In [ ]:
# segment small round structures with this (golgi vesicles)
dot_scale_1 = 1.6
dot_cut_1 = 0.02

dot_scale_2 = 0
dot_cut_2 = 0

dot_scale_3 = 0
dot_cut_3 = 0

dot_method = '3D'

bw_extra = dot_filter_3(struct_img,
                        dot_scale_1,
                        dot_cut_1,
                        dot_scale_2,
                        dot_cut_2,
                        dot_scale_3,
                        dot_cut_3,
                        dot_method)

## **`STEP 6` - Combine Segmentations (logical or)**

- combine the two segmentations with logical *OR*

In [ ]:
# combine the two segmentations together
bw_combine = np.logical_or(bw_extra, bw_thin)

# ***POST-PROCESSING prototype - golgi***

## **`STEP 7` - Remove small holes and objects**

- fill holes (hole size = user input)
- remove small objects (object size = user input)
- filter method (method = user input)

In [ ]:
###################
# POST_PROCESSING
###################
hole_min_width = 0
hole_max_width = 1

small_object_width = 3

fill_filter_method = "3D"

cleaned_img2 = fill_and_filter_linear_size(bw_combine, 
                                           hole_min=hole_min_width, 
                                           hole_max=hole_max_width, 
                                           min_size=small_object_width,
                                           method=fill_filter_method)

# ***POST-POST-PROCESSING prototype - golgi***

## **`STEP 8` - Label objects**

- label unique golgi objects based on connectivity

In [ ]:
###################
# LABELING
###################
golgi_labels = label_uint16(cleaned_img2)

In [ ]:
golgi_labels.dtype

## **Visualize with `napari` 1**
Visualize the first-pass segmentation and labeling with `napari`.

In [ ]:
viewer = napari.Viewer()

In [ ]:
viewer.add_image(
    struct_img)

viewer.add_image(
    cleaned_img2,
    opacity=0.3,)

viewer.add_labels(
    golgi_labels,
    opacity=0.3)

# ***EXTRACTION prototype - golgi***

- save labeled ***golgi*** (golgi, GL) as unsigned integer 16-bit tif files

In [ ]:
out_file_n = export_inferred_organelle(golgi_labels, "golgi", meta_dict, out_data_path)

# **Define `_infer_golgi` function**

> ###### 📝 **these functions mainly serve for downstream prototyping in the notebooks. Each step above has an independent function that is implemented in the plugin for easy of use**

In [ ]:
##########################
#  infer_golgi
##########################
def _infer_golgi(
            in_img: np.ndarray,
            golgi_ch: int,
            median_sz: int,
            gauss_sig: float,
            mo_method: str,
            mo_adjust: float,
            mo_cutoff_size: int,
            min_thickness: int,
            thin_dist: int,
            dot_scale_1: float,
            dot_cut_1: float,
            dot_scale_2: float,
            dot_cut_2: float,
            dot_scale_3: float,
            dot_cut_3: float,
            dot_method: str,
            min_hole_w: int,
            max_hole_w: int,
            small_obj_w: int,
            fill_filter_method: str
        ) -> np.ndarray:

    """
    Procedure to infer golgi from linearly unmixed input.

   Parameters
    ------------
    in_img: 
        a 3d image containing all the channels
    median_sz: 
        width of median filter for signal
    mo_method: 
         which method to use for calculating global threshold. Options include:
         "triangle" (or "tri"), "median" (or "med"), and "ave_tri_med" (or "ave").
         "ave" refers the average of "triangle" threshold and "mean" threshold.
    mo_adjust: 
        Masked Object threshold `local_adjust`
    mo_cutoff_size: 
        Masked Object threshold `size_min`
    min_thinkness: 
        Half of the minimum width you want to keep from being thinned.
        For example, when the object width is smaller than 4, you don't
        want to make this part even thinner (may break the thin object
        and alter the topology), you can set this value as 2.
    thin_dist: 
        the amount to thin (has to be an positive integer). The number of
         pixels to be removed from outter boundary towards center.
    dot_scale: 
        scales (log_sigma) for dot filter (1,2, and 3)
    dot_cut: 
        threshold for dot filter thresholds (1,2,and 3)
    small_obj_w: 
        minimu object size cutoff for nuclei post-processing
    
    Returns
    -------------
    golgi_object
        mask defined extent of golgi object
    """

    ###################
    # EXTRACT
    ###################    
    golgi = select_channel_from_raw(in_img, golgi_ch)

    ###################
    # PRE_PROCESSING
    ###################    
    golgi =  scale_and_smooth(golgi,
                              median_size = median_sz, 
                              gauss_sigma = gauss_sig)
    ###################
    # CORE_PROCESSING
    ###################
    bw = masked_object_thresh(golgi, global_method=mo_method, cutoff_size=mo_cutoff_size, local_adjust=mo_adjust)

    bw_thin = topology_preserving_thinning(bw, min_thickness, thin_dist)

    bw_extra = dot_filter_3(golgi, dot_scale_1, dot_cut_1, dot_scale_2, dot_cut_2, dot_scale_3, dot_cut_3, dot_method)

    bw = np.logical_or(bw_extra, bw_thin)
    ###################
    # POST_PROCESSING
    ###################
    struct_obj = fill_and_filter_linear_size(bw, 
                                             hole_min=min_hole_w, 
                                             hole_max=max_hole_w, 
                                             min_size=small_obj_w,
                                             method=fill_filter_method)

    ###################
    # LABELING
    ###################
    struct_obj1 = label_uint16(struct_obj)

    return struct_obj1


# **Test `_infer_golgi` function**

In [ ]:
_golgi_object =  _infer_golgi(
    img_data,
    GOLGI_CH,
    median_sz,
    gauss_sig,
    thresh_method,
    thresh_adj,
    cell_wise_min_area,
    thin_dist_preserve,
    thin_dist,
    dot_scale_1,
    dot_cut_1,
    dot_scale_2,
    dot_cut_2,
    dot_scale_3,
    dot_cut_3,
    dot_method,
    hole_min_width,
    hole_max_width,
    small_object_width,
    fill_filter_method) 

_golgi_object.dtype

In [ ]:
np.all(golgi_labels == _golgi_object)

## **Visualize with `napari` 2**

In [ ]:
viewer.add_image(
    _golgi_object,
    opacity=0.3)

In [ ]:
nbscreenshot(viewer, canvas_only=True)

In [ ]:
viewer.close()

-------------
### NEXT: INFER PEROXISOME

proceed to [1.5_infer_peroxisome.ipynb](./1.5_infer_peroxisome.ipynb)
